In [1]:
# 01 - Import libraries and configuration
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import (
    ResNet50, EfficientNetB6, ResNet152V2, DenseNet201, InceptionResNetV2
)
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as inception_preprocess
import os
import gc
import json
import glob
import time
from tqdm.auto import tqdm
from PIL import Image
try:
    import pynvml
    PYNVML_AVAILABLE = True
except ImportError:
    PYNVML_AVAILABLE = False
    print("Warning: pynvml not available. Energy tracking disabled.")

CONFIG = {
    'BATCH_SIZE': 64,
    'CALIBRATION_BATCH_SIZE': 128,
    'RANDOM_STATE': 42,
}

DATASET_PATHS = {
    'HAM10000': '/kaggle/input/skin-cancer-mnist-ham10000',
    'ISIC2019': '/kaggle/input/isic-2019-skin-lesion-images-for-classification',
    'PAD-UFES-20': '/kaggle/input/skin-cancer',
    'BCN20000': '/kaggle/input/datasets/pasutchien/bcn20000',
}

PROCESS_DATASETS = ['HAM10000', 'PAD-UFES-20', 'BCN20000']

HEAVY_MODELS = {
    'ResNet50': {
        'model_fn': ResNet50,
        'input_size': (224, 224),
        'preprocess': resnet50_preprocess
    },
    'EfficientNetB6': {
        'model_fn': EfficientNetB6,
        'input_size': (528, 528),
        'preprocess': efficientnet_preprocess
    },
    'ResNet152V2': {
        'model_fn': ResNet152V2,
        'input_size': (224, 224),
        'preprocess': resnet_preprocess
    },
    'DenseNet201': {
        'model_fn': DenseNet201,
        'input_size': (224, 224),
        'preprocess': densenet_preprocess
    },
    'InceptionResNetV2': {
        'model_fn': InceptionResNetV2,
        'input_size': (299, 299),
        'preprocess': inception_preprocess
    }
}

print("Heavy models configuration loaded")

2026-02-20 17:40:50.399649: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771609250.609248      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771609250.668929      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771609251.174468      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771609251.174515      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771609251.174518      24 computation_placer.cc:177] computation placer alr

Heavy models configuration loaded


In [2]:
# 02 - Dataset handler functions
def find_image_paths(dataset_name, base_path):
    image_paths = {}
    all_image_files = []
    
    if dataset_name == 'BCN20000':
        # BCN20000 images are split across two subdirectories
        search_dirs = [
            os.path.join(base_path, 'BCN_20k_train', 'bcn_20k_train'),
            os.path.join(base_path, 'BCN_20k_test', 'bcn_20k_test'),
        ]
    else:
        search_dirs = [base_path]
    
    # Search for both .jpg and .png files (PAD-UFES-20 uses .png)
    for search_dir in search_dirs:
        for ext in ['*.jpg', '*.png']:
            for img_file in glob.glob(os.path.join(search_dir, '**', ext), recursive=True):
                all_image_files.append(img_file)
    
    all_image_files = sorted(all_image_files)
    
    for img_file in all_image_files:
        image_id = os.path.splitext(os.path.basename(img_file))[0]
        if image_id not in image_paths:
            image_paths[image_id] = img_file
    
    return image_paths

def load_dataset_metadata(dataset_name, base_path):
    if dataset_name == 'HAM10000':
        metadata_path = os.path.join(base_path, 'HAM10000_metadata.csv')
        if os.path.exists(metadata_path):
            metadata = pd.read_csv(metadata_path)
            if 'image_id' in metadata.columns:
                metadata = metadata.dropna(subset=['image_id'])
                return metadata
    elif dataset_name == 'ISIC2019':
        metadata_path = os.path.join(base_path, 'ISIC_2019_Training_Metadata.csv')
        if os.path.exists(metadata_path):
            metadata = pd.read_csv(metadata_path)
            if 'image' in metadata.columns:
                metadata = metadata.rename(columns={'image': 'image_id'})
                metadata = metadata.dropna(subset=['image_id'])
                return metadata
    elif dataset_name == 'PAD-UFES-20':
        metadata_path = os.path.join(base_path, 'metadata.csv')
        if os.path.exists(metadata_path):
            metadata = pd.read_csv(metadata_path)
            # PAD-UFES-20: try multiple possible column names for image identifier
            if 'img_id' in metadata.columns:
                metadata = metadata.rename(columns={'img_id': 'image_id'})
            elif 'image_id' in metadata.columns:
                pass  # Already has image_id
            elif 'image' in metadata.columns:
                metadata = metadata.rename(columns={'image': 'image_id'})
            elif 'filename' in metadata.columns:
                metadata = metadata.rename(columns={'filename': 'image_id'})
            else:
                # Try to find any column that might contain image identifiers
                # Check first few values to see if any column looks like image IDs
                for col in metadata.columns:
                    sample_val = str(metadata[col].iloc[0]) if len(metadata) > 0 else ''
                    if 'PAT_' in sample_val or sample_val.endswith('.png') or sample_val.endswith('.jpg'):
                        metadata = metadata.rename(columns={col: 'image_id'})
                        break
            
            if 'image_id' in metadata.columns:
                # Normalize image_id: remove file extensions, strip whitespace, convert to string
                metadata['image_id'] = metadata['image_id'].astype(str).str.replace('.png', '', regex=False).str.replace('.jpg', '', regex=False).str.replace('.jpeg', '', regex=False).str.strip()
                metadata = metadata.dropna(subset=['image_id'])
                metadata = metadata[metadata['image_id'] != '']
                return metadata
    elif dataset_name == 'BCN20000':
        # Load and concatenate train and test CSVs
        train_csv = os.path.join(base_path, 'bcn_20k_train.csv')
        test_csv  = os.path.join(base_path, 'bcn_20k_test.csv')
        parts = []
        for csv_path in [train_csv, test_csv]:
            if os.path.exists(csv_path):
                df = pd.read_csv(csv_path)
                parts.append(df)
            else:
                print(f"  Warning: BCN20000 CSV not found: {csv_path}")
        
        if not parts:
            return None
        
        metadata = pd.concat(parts, ignore_index=True)
        
        # Normalize image identifier column
        for col in ['bcn_filename', 'image', 'isic_id', 'image_id', 'filename']:
            if col in metadata.columns:
                metadata = metadata.rename(columns={col: 'image_id'})
                break
        
        if 'image_id' not in metadata.columns:
            print(f"  Warning: Could not find image identifier column in BCN20000 metadata.")
            print(f"  Available columns: {list(metadata.columns)}")
            return None
        
        # Normalize image_id: strip extensions and whitespace
        metadata['image_id'] = (metadata['image_id'].astype(str)
                                .str.replace('.jpg', '', regex=False)
                                .str.replace('.png', '', regex=False)
                                .str.replace('.jpeg', '', regex=False)
                                .str.strip())
        metadata = metadata.dropna(subset=['image_id'])
        metadata = metadata[metadata['image_id'] != '']
        metadata = metadata.drop_duplicates(subset=['image_id'])
        return metadata
    
    return None

print("Dataset handler functions defined")

Dataset handler functions defined


In [3]:
# 03 - RAM caching function
def load_images_to_ram_cache(image_paths_dict, image_ids_list, target_size):
    image_cache = {}
    valid_image_ids = []
    
    for image_id in tqdm(image_ids_list, desc=f"Caching images at {target_size}", unit="img"):
        if image_id in image_cache:
            continue
        
        if image_id not in image_paths_dict:
            continue
        
        img_path = image_paths_dict[image_id]
        if not os.path.exists(img_path):
            continue
        
        try:
            img = Image.open(img_path).convert('RGB')
            img = img.resize(target_size)
            img_array = np.array(img, dtype=np.uint8)
            image_cache[image_id] = img_array
            valid_image_ids.append(image_id)
        except Exception as e:
            print(f"  Warning: Failed to load {image_id}: {e}")
            continue
    
    return image_cache, valid_image_ids

print("RAM caching function defined")

RAM caching function defined


In [4]:
# 04 - Energy calibration function (Direct pynvml Power Tracking)
def calibrate_energy(model, image_cache, image_ids, preprocess_fn, calibration_size=32, model_name=None, output_dir=None, num_repeats=50):
    if not PYNVML_AVAILABLE or len(image_ids) < 8:
        return None
    
    calibration_sizes = [calibration_size, 16, 8, 4]
    calibration_sizes = [s for s in calibration_sizes if len(image_ids) >= s]
    
    for cal_size in calibration_sizes:
        try:
            calibration_ids = image_ids[:cal_size]
            calibration_images_uint8 = []
            for img_id in calibration_ids:
                calibration_images_uint8.append(image_cache[img_id])
            
            calibration_images_uint8 = np.array(calibration_images_uint8, dtype=np.uint8)
            calibration_images_float32 = calibration_images_uint8.astype(np.float32)
            calibration_images_processed = preprocess_fn(calibration_images_float32)
            
            pynvml.nvmlInit()
            handle = pynvml.nvmlDeviceGetHandleByIndex(0)
            
            _ = model(calibration_images_processed, training=False)
            
            start_time = time.time()
            power_readings = []
            
            for _ in range(num_repeats):
                _ = model(calibration_images_processed, training=False)
                power = pynvml.nvmlDeviceGetPowerUsage(handle)
                power_readings.append(power / 1000.0)
            
            duration = time.time() - start_time
            avg_power_watts = sum(power_readings) / len(power_readings)
            total_joules = avg_power_watts * duration
            
            total_samples = num_repeats * cal_size
            joules_per_sample = total_joules / total_samples
            
            del calibration_images_uint8, calibration_images_float32, calibration_images_processed
            gc.collect()
            
            if cal_size < calibration_size:
                print(f"    Note: Used smaller batch size ({cal_size}) due to memory constraints")
            
            return joules_per_sample
            
        except Exception as e:
            if "OOM" in str(e) or "out of memory" in str(e).lower():
                if cal_size == calibration_sizes[-1]:
                    print(f"  Calibration failed: OOM even with batch size {cal_size}")
                    return None
                continue
            else:
                print(f"  NVML Error: {e}")
                if cal_size == calibration_sizes[-1]:
                    return None
                continue
    
    return None

print("Energy calibration function defined")

Energy calibration function defined


In [5]:
# 05 - Feature extraction function (Batch-wise to prevent OOM)
def extract_features(model, image_cache, image_ids, preprocess_fn, batch_size=32):
    features_list = []
    num_samples = len(image_ids)
    failed_batches = 0
    
    for i in tqdm(range(0, num_samples, batch_size), desc="Extracting features", unit="batch"):
        batch_end = min(i + batch_size, num_samples)
        batch_ids = image_ids[i:batch_end]
        
        try:
            batch_images_uint8 = []
            for img_id in batch_ids:
                if img_id in image_cache:
                    batch_images_uint8.append(image_cache[img_id])
                else:
                    continue
            
            if len(batch_images_uint8) == 0:
                failed_batches += 1
                continue
            
            batch_images_uint8 = np.array(batch_images_uint8, dtype=np.uint8)
            batch_images_float32 = batch_images_uint8.astype(np.float32)
            batch_images_processed = preprocess_fn(batch_images_float32)
            
            batch_features = model.predict_on_batch(batch_images_processed)
            features_list.append(batch_features)
            
            del batch_images_uint8, batch_images_float32, batch_images_processed, batch_features
            gc.collect()
        except Exception as e:
            failed_batches += 1
            print(f"  Warning: Batch failed (indices {i}-{batch_end-1})")
            continue
    
    if failed_batches > 0:
        print(f"  Warning: {failed_batches} batches failed during extraction")
    
    if len(features_list) == 0:
        raise ValueError("No features extracted - all batches failed")
    
    features = np.vstack(features_list)
    return features

print("Feature extraction function defined")

Feature extraction function defined


In [6]:
# 06 - Main execution loop (Multi-dataset processing)
output_base_dir = './output'

for dataset_name in PROCESS_DATASETS:
    print(f"\n{'='*70}")
    print(f"DATASET: {dataset_name}")
    print(f"{'='*70}")
    
    if dataset_name not in DATASET_PATHS:
        print(f"  ERROR: Dataset {dataset_name} not in DATASET_PATHS. Skipping.")
        continue
    
    base_path = DATASET_PATHS[dataset_name]
    if not os.path.exists(base_path):
        print(f"  ERROR: Dataset path not found: {base_path}. Skipping.")
        continue
    
    print(f"  Base path: {base_path}")
    
    metadata = load_dataset_metadata(dataset_name, base_path)
    if metadata is None:
        print(f"  ERROR: Could not load metadata for {dataset_name}. Skipping.")
        continue
    
    print(f"  Metadata loaded: {len(metadata)} samples")
    
    image_paths = find_image_paths(dataset_name, base_path)
    print(f"  Images found: {len(image_paths)} files")
    
    if len(image_paths) == 0:
        print(f"  ERROR: No images found for {dataset_name}. Skipping.")
        continue
    
    # Normalize metadata image_ids for comparison (already normalized in load_dataset_metadata, but ensure consistency)
    metadata_ids_normalized = set(metadata['image_id'].astype(str).str.strip().values)
    image_ids_normalized = set(image_paths.keys())
    
    common_ids = list(metadata_ids_normalized & image_ids_normalized)
    print(f"  Common IDs (metadata + images): {len(common_ids)}")
    
    if len(common_ids) == 0:
        # Debug: show sample IDs from both sources
        print(f"  DEBUG: Sample metadata IDs (first 5): {list(metadata_ids_normalized)[:5]}")
        print(f"  DEBUG: Sample image IDs (first 5): {list(image_ids_normalized)[:5]}")
        print(f"  ERROR: No common IDs between metadata and images. Skipping.")
        continue
    
    resolution_groups = {}
    for model_name, model_config in HEAVY_MODELS.items():
        resolution = model_config['input_size']
        if resolution not in resolution_groups:
            resolution_groups[resolution] = []
        resolution_groups[resolution].append((model_name, model_config))
    
    print(f"\n  Models grouped into {len(resolution_groups)} resolution groups")
    
    for resolution_idx, (resolution, models_in_group) in enumerate(resolution_groups.items(), 1):
        print(f"\n  {'-'*70}")
        print(f"  RESOLUTION GROUP {resolution_idx}/{len(resolution_groups)}: {resolution}")
        print(f"  Models: {[m[0] for m in models_in_group]}")
        print(f"  {'-'*70}")
        
        print(f"  Loading cache for resolution {resolution}...")
        image_cache, valid_image_ids = load_images_to_ram_cache(image_paths, common_ids, resolution)
        print(f"  Cache loaded: {len(image_cache)} images\n")
        
        if len(image_cache) == 0:
            print(f"  WARNING: No images cached. Skipping this resolution group.")
            continue
        
        for model_idx, (model_name, model_config) in enumerate(models_in_group, 1):
            print(f"\n  Processing Model {model_idx}/{len(models_in_group)}: {model_name}")
            
            try:
                print(f"    Loading model with ImageNet weights...")
                base_model = model_config['model_fn'](
                    weights='imagenet',
                    include_top=False,
                    input_shape=(*model_config['input_size'], 3),
                    pooling='avg'
                )
                
                output_dir = os.path.join(output_base_dir, dataset_name, model_name)
                os.makedirs(output_dir, exist_ok=True)
                
                joules_per_sample = None
                if PYNVML_AVAILABLE:
                    print(f"    Energy calibration phase (32 samples, 50 repeats)...")
                    joules_per_sample = calibrate_energy(
                        base_model,
                        image_cache,
                        valid_image_ids,
                        model_config['preprocess'],
                        calibration_size=32,
                        model_name=model_name,
                        output_dir=output_dir,
                        num_repeats=50
                    )
                    if joules_per_sample:
                        print(f"    Calibrated: {joules_per_sample:.6f} Joules per sample")
                    else:
                        print(f"    Warning: Energy calibration failed - energy stats will be N/A")
                
                print(f"    Extracting features (batch_size={32})...")
                features = extract_features(
                    base_model,
                    image_cache,
                    valid_image_ids,
                    model_config['preprocess'],
                    batch_size=32
                )
                
                features_path = os.path.join(output_dir, 'features.npy')
                ids_path = os.path.join(output_dir, 'ids.npy')
                energy_path = os.path.join(output_dir, 'energy_stats.json')
                
                np.save(features_path, features)
                np.save(ids_path, np.array(valid_image_ids))
                
                energy_stats = {
                    'model_name': model_name,
                    'dataset_name': dataset_name,
                    'resolution': resolution,
                    'num_samples': len(valid_image_ids),
                    'feature_dim': features.shape[1],
                    'joules_per_sample': joules_per_sample if joules_per_sample else None,
                    'total_energy_joules': joules_per_sample * len(valid_image_ids) if joules_per_sample else None
                }
                
                with open(energy_path, 'w') as f:
                    json.dump(energy_stats, f, indent=2)
                
                print(f"    Saved {features.shape} features for {model_name}")
                print(f"    Features: {features_path}")
                print(f"    IDs: {ids_path}")
                print(f"    Energy stats: {energy_path}")
                
                print(f"\n{'='*55}")
                print(f"FINAL PROCESSING SUMMARY for {model_name} on {dataset_name}")
                print(f"{'='*55}")
                print(f"> Input Files Found:  {len(image_paths)}")
                print(f"> Features Extracted: {features.shape}")
                print(f"> Energy/Sample:      {joules_per_sample:.6f} J" if joules_per_sample else "> Energy/Sample:      N/A")
                print(f"> Saved to:           {output_dir}")
                print(f"{'='*55}\n")
                
                del base_model, features
                gc.collect()
                tf.keras.backend.clear_session()
                gc.collect()
                
            except Exception as e:
                print(f"    ERROR: {model_name} - {str(e)}")
                import traceback
                traceback.print_exc()
                gc.collect()
                tf.keras.backend.clear_session()
                continue
        
        print(f"\n  Clearing cache for resolution {resolution}...")
        del image_cache, valid_image_ids
        gc.collect()
        print(f"  Memory freed for resolution group {resolution}\n")
    
    print(f"  Dataset {dataset_name} processing complete\n")

print(f"\n{'='*70}")
print(f"FEATURE EXTRACTION COMPLETE: All datasets and models processed")
print(f"{'='*70}\n")


DATASET: HAM10000
  Base path: /kaggle/input/skin-cancer-mnist-ham10000
  Metadata loaded: 10015 samples
  Images found: 10015 files
  Common IDs (metadata + images): 10015

  Models grouped into 3 resolution groups

  ----------------------------------------------------------------------
  RESOLUTION GROUP 1/3: (224, 224)
  Models: ['ResNet50', 'ResNet152V2', 'DenseNet201']
  ----------------------------------------------------------------------
  Loading cache for resolution (224, 224)...


Caching images at (224, 224):   0%|          | 0/10015 [00:00<?, ?img/s]

  Cache loaded: 10015 images


  Processing Model 1/3: ResNet50
    Loading model with ImageNet weights...


I0000 00:00:1771609446.833344      24 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
    Energy calibration phase (32 samples, 50 repeats)...


...

    Saved (18914, 2304) features for EfficientNetB6
    Features: ./output/BCN20000/EfficientNetB6/features.npy
    IDs: ./output/BCN20000/EfficientNetB6/ids.npy
    Energy stats: ./output/BCN20000/EfficientNetB6/energy_stats.json

FINAL PROCESSING SUMMARY for EfficientNetB6 on BCN20000
> Input Files Found:  18914
> Features Extracted: (18914, 2304)
> Energy/Sample:      9.473756 J
> Saved to:           ./output/BCN20000/EfficientNetB6


  Clearing cache for resolution (528, 528)...
  Memory freed for resolution group (528, 528)


  ----------------------------------------------------------------------
  RESOLUTION GROUP 3/3: (299, 299)
  Models: ['InceptionResNetV2']
  ----------------------------------------------------------------------
  Loading cache for resolution (299, 299)...


Caching images at (299, 299):   0%|          | 0/18914 [00:00<?, ?img/s]

  Cache loaded: 18914 images


  Processing Model 1/1: InceptionResNetV2
    Loading model with ImageNet weights...
    Energy calibration phase (32 samples, 50 repeats)...
    Calibrated: 1.363509 Joules per sample
    Extracting features (batch_size=32)...


Extracting features:   0%|          | 0/592 [00:00<?, ?batch/s]

    Saved (18914, 1536) features for InceptionResNetV2
    Features: ./output/BCN20000/InceptionResNetV2/features.npy
    IDs: ./output/BCN20000/InceptionResNetV2/ids.npy
    Energy stats: ./output/BCN20000/InceptionResNetV2/energy_stats.json

FINAL PROCESSING SUMMARY for InceptionResNetV2 on BCN20000
> Input Files Found:  18914
> Features Extracted: (18914, 1536)
> Energy/Sample:      1.363509 J
> Saved to:           ./output/BCN20000/InceptionResNetV2


  Clearing cache for resolution (299, 299)...
  Memory freed for resolution group (299, 299)

  Dataset BCN20000 processing complete


FEATURE EXTRACTION COMPLETE: All datasets and models processed

